In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql.functions import sum

In [2]:
spark = SparkSession.builder.appName("BD Lab").getOrCreate()

24/11/07 13:34:02 WARN Utils: Your hostname, snucse-HP-Pro-Tower-400-G9-PCI-Desktop-PC resolves to a loopback address: 127.0.1.1; using 10.23.22.95 instead (on interface eno1)
24/11/07 13:34:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/07 13:34:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data1 = spark.read.option("header", True).csv("data1.csv")
data2 = spark.read.option("header", True).csv("data2.csv")

In [4]:
data1.show(5)

+---------+----------+--------+-----------+---------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|
+---------+----------+--------+-----------+---------+------+
|        1|  15634602|Hargrave|        619|   France|Female|
|        2|  15647311|    Hill|        608|    Spain|Female|
|        3|  15619304|    Onio|        502|   France|Female|
|        4|  15701354|    Boni|        699|   France|Female|
|        5|  15737888|Mitchell|        850|    Spain|Female|
+---------+----------+--------+-----------+---------+------+
only showing top 5 rows



In [5]:
data2.show(5)

+----------+---+------+---------+-------------+---------+--------------+---------------+------+
|CustomerId|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+----------+---+------+---------+-------------+---------+--------------+---------------+------+
|  15634602| 42|     2|        0|            1|        1|             1|      101348.88|     1|
|  15647311| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|  15619304| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|  15701354| 39|     1|        0|            2|        0|             0|       93826.63|     0|
|  15737888| 43|     2|125510.82|            1|        1|             1|        79084.1|     0|
+----------+---+------+---------+-------------+---------+--------------+---------------+------+
only showing top 5 rows



In [6]:
joined_data = data1.join(data2, on="CustomerID", how="inner")

In [7]:
active_avg_salary = joined_data.filter(joined_data["IsActiveMember"] == 1).select(avg("EstimatedSalary").alias("AverageSalary"))

In [8]:
print("Average Salary : ",active_avg_salary.collect()[0][0])

Average Salary :  99452.96589400139


In [9]:
average_age_by_gender = joined_data.groupBy("Gender").agg(avg("Age").alias("AverageAge"))

average_age_by_gender.show()

+------+------------------+
|Gender|        AverageAge|
+------+------------------+
|Female|39.238388729914156|
|  Male|38.658237126626354|
+------+------------------+



In [10]:
joined_data.createOrReplaceTempView("customer_data")

In [11]:
high_earners = spark.sql("SELECT CustomerId, Surname, EstimatedSalary FROM customer_data WHERE EstimatedSalary > 100000")

high_earners.show()

+----------+-------------+---------------+
|CustomerId|      Surname|EstimatedSalary|
+----------+-------------+---------------+
|  15634602|     Hargrave|      101348.88|
|  15647311|         Hill|      112542.58|
|  15619304|         Onio|      113931.57|
|  15574012|          Chu|      149756.71|
|  15656148|       Obinna|      119346.88|
|  15691483|         Chin|      190857.79|
|  15661507|      Muldrow|      158684.81|
|  15577657|     McDonald|      170886.17|
|  15597945|     Dellucci|      138555.46|
|  15699309|    Gerasimov|      118913.53|
|  15625047|          Yen|      187616.16|
|  15738191|      Maclean|      124508.29|
|  15736816|        Young|      170041.95|
|  15728693|   McWilliams|      100187.43|
|  15589475|      Azikiwe|      140469.38|
|  15706552|Odinakachukwu|      156731.91|
|  15732963|     Clements|      142033.07|
|  15788448|       Watson|      114066.77|
|  15585768|      Cameron|      178074.04|
|  15738148|       Clarke|      181297.65|
+----------

In [12]:
products_by_geography = joined_data.groupBy("Geography").agg(sum("NumOfProducts").alias("TotalProducts"))

In [13]:
products_by_geography.show()

+---------+-------------+
|Geography|TotalProducts|
+---------+-------------+
|  Germany|       3813.0|
|   France|       7676.0|
|    Spain|       3813.0|
+---------+-------------+

